In [1]:
import cv2
import numpy as np
from PIL import Image

In [2]:
#Take input images : img_grey = grey scale image & img = RGB image
img_grey = np.asarray(Image.open('lena.png').convert('L'))
img=np.asarray(Image.open('wolves.png'))
r=img[:,:,0]
g=img[:,:,1]
b=img[:,:,2]

In [3]:
# Question 1 A

In [4]:
#Padding Function 

"""
returns new array of image with paddded values
"""
def padding_test(img,p):
  h=img.shape[0]
  
  w=img.shape[1]
  #padding value
  #minimum padding value must be 2
  pad_value = 100
  add = int(pad_value/2)
  

  new_array = np.zeros((h+pad_value,w+pad_value),dtype='float32')
  [h1,w1]=new_array.shape
  
 
  for i in range(0,h):
      for j in range(0,w):
        new_array[i+add,j+add] = img[i,j]
  if p == 'zero':
    return new_array
  elif p == 'copy': 
    for i in range(0,h):
        for j in range(0,w):
            new_array[0:add,0:add]=img[0,0]
            new_array[-add:-1,0:add]=img[h-1,0]
            new_array[0:add,-add:-1]=img[0,w-1]
            new_array[-add:-1,-add:-1]=img[h-1,w-1]
            
            new_array[0:add,add:w1-add]=img[0,0:w]
            new_array[-add:-1,add:w1-add]=img[h-1,0:w]
            new_array[add:h1-add,1:add]=img[0:h,0:1]
            new_array[add:h1-add,-add:-1]=img[0:h,w-1:w]
    
             
                    
    return new_array
  elif p == 'wrap': 
    for i in range(0,h):
        for j in range(0,w):
            new_array[0:add,0:add]=img[h-add:h,w-add:w]
            new_array[-add:-1,0:add]=img[0:add-1,w-add:w]
            new_array[0:add,-add:-1]=img[h-add:h,0:add-1]
            new_array[-add:-1,-add:-1]=img[0:add-1,0:add-1]
            
            new_array[0:add,add:w1-add]=img[h-add:h,0:w]
            new_array[-add:-1,add:w1-add]=img[0:add-1,0:w]
            new_array[add:h1-add,1:add+1]=img[0:h,w-add:w]
            new_array[add:h1-add,-add:-1]=img[0:h,1:add]
                            
    return new_array

  elif p == 'reflect':
        
    for i in range(0,h):
        for j in range(0,w):
            new_array[0:add,0:add]=img[add-1::-1,add-1::-1]
            new_array[-add:-1,0:add]=img[h:h-add:-1,add-1::-1]
            new_array[0:add,-add:-1]=img[add-1::-1,w:w-add:-1]
            new_array[-add:-1,-add:-1]=img[h-1:h-add:-1,w-1:w-add:-1]

            new_array[0:add,add:w1-add]=img[add-1::-1,0:w]
            new_array[-add:-1,add:w1-add]=img[h:h-add:-1,0:w]
            new_array[add:h1-add,1:add+1]=img[0:h+1,add-1::-1]
            new_array[add:h1-add,-add:-1]=img[0:h+1,w:w-add:-1]
    return new_array
 
  

In [22]:
#grey image padding call
"""define the type of padding desired in pad = '___' """
pad='copy'
x=padding_test(img_grey,pad)
y=Image.fromarray(x)
y.show()

In [467]:
#padding RGB image
"""split rgb image into channels and 
perform padding and again merge the output images from each channel"""
pad='reflect'
x=padding_test(r,pad)
r_img = Image.fromarray(x)
r_img=r_img.convert('L')
y=padding_test(g,pad)
g_img = Image.fromarray(y)
g_img=g_img.convert('L')
z=padding_test(b,pad)
b_img = Image.fromarray(z)
b_img=b_img.convert('L')
img = Image.merge('RGB',(r_img,g_img,b_img))
img.show()


In [6]:
#kernels 
kernel_prewitt_x = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
kernel_prewitt_y = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernel_sobel_x = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
kernel_sobel_y = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
kernel_roberts_x = np.array([[0,1],[-1,0]])
kernel_roberts_y = np.array([[1,0],[0,-1]])
kernel_box = np.array([[1,1,1],[1,1,1],[1,1,1]])
kernel_box = 1/9*kernel_box
kernel_first_order_x = np.array([[1,-1]])
kernel_first_order_y = np.array([[-1],[1]])


In [7]:
#convolution function : displays the output image after convolution
def conv(f,w,pad):
  
  current_matrix = np.array([[0,0,0],[0,0,0],[0,0,0]])

  x = padding_test(f,pad)
  [s,t] = w.shape

  [a,b] = x.shape
  convoluted_img = np.zeros(shape=(a,b),dtype='float32')
  
  for i in range(0,a-2):
    for j in range(0,b-2):
      current_matrix = x[i:i+s,j:j+t]
      convoluted_img[i,j] = (np.multiply(w,current_matrix)).sum()
      convoluted_img = convoluted_img.astype(np.int16)
  res = Image.fromarray(convoluted_img)
  res.show()
   

In [16]:
#modified convolution for RGB channel convolutin
def conv_1(f,w,pad):
  
  current_matrix = np.array([[0,0,0],[0,0,0],[0,0,0]])

  x = padding_test(f,pad)
  [s,t] = w.shape

  [a,b] = x.shape
  convoluted_img = np.zeros(shape=(a,b),dtype='float32')
  
  for i in range(0,a-2):
    for j in range(0,b-2):
      current_matrix = x[i:i+s,j:j+t]
      convoluted_img[i,j] = (np.multiply(w,current_matrix)).sum()
      convoluted_img = convoluted_img.astype(np.int16)
  res = Image.fromarray(convoluted_img)
  return convoluted_img

In [485]:
#performing convolution call for grey scale 
"""performed on Lena.png"""
pad='reflect'
x=conv(img_grey,kernel_first_order_y,pad)


In [17]:
#convolution call for RGB scale
"""
performed on wolves.png
"""
pad='reflect'
img = Image.open('wolves.png')
x=Image.Image.split(img)
r=np.array(x[0])
g=np.array(x[1])
b=np.array(x[2])
r1=conv_1(r,kernel_box,pad)
r_img = Image.fromarray(r1).convert('L')
g1=conv_1(g,kernel_box,pad)
g_img = Image.fromarray(g1).convert('L')
b1=conv_1(b,kernel_box,pad)
b_img = Image.fromarray(b1).convert('L')
img = Image.merge('RGB',(r_img,g_img,b_img))
img.show()

In [9]:
# Question 1 B

In [18]:
# Generate grey scale image of size 1024x1024 
"""
perform convolution on generated grey scale matrix using same convolution function
"""
grey_img=np.zeros((1024,1024),dtype='int32')
grey_img[512,512]=255
g_op=Image.fromarray(grey_img)
g_op.show()
pad='zero'
conv(grey_img,kernel_sobel_x,pad)